# Cars_24 scraping

In [ ]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException

#Importing requests
import requests

# importing regex
import re

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.keys import Keys

#Initializing driver
driver=webdriver.Chrome('I:\chromedriver.exe')
driver.maximize_window()

#Loading website
url='https://www.cars24.com/buy-used-cars/'
driver.get(url)
time.sleep(2)

#Closing location
close_popup=driver.find_element_by_xpath('//div[@class="float-right TBxAN"]').click()
time.sleep(2)

#Seeing all cars
view_all=driver.find_element_by_xpath('//div[@class="_3tB_u"]/button').click()
time.sleep(15)

#Details to scrape
car_type=[]
location=[]
brand=[]
model=[]
version=[]
year=[]
fuel_type=[]
transmission_type=[]
distance_driven=[]
price=[]

#Location selection
location_dropdown=driver.find_element_by_xpath('//p[@class="_1OR7d"]').click()
time.sleep(10)


count=1
while count<11:
    popular_cities=driver.find_element_by_xpath(f'//ul[@class="_16Bvy"]/li[{count}]').click()
    time.sleep(10)
    city=driver.find_element_by_xpath('//p[@class="_1OR7d"]').text
    print('City selected: ',city)
    
    #Getting details of cars from each city
    i=1
    while i<6:
        #Filtering the body type
        filters=driver.find_element_by_xpath('//ul[@class="_1bWR_"]/li[6]')
        filters.click()
        time.sleep(2)
    
        #Scrolling to interact with the body type
        select=driver.find_element_by_xpath(f'//ul[@class="sVtBG _7AhJm"]/li[{i}]')
        actions = ActionChains(driver)
        actions.move_to_element(select).perform()
        select.click()
        time.sleep(5)
    
        body_type_selected = driver.find_element_by_xpath(f'//ul[@class="sVtBG _7AhJm"]/li[{i}]/div').text
        print(f'  Selected {body_type_selected}')
        time.sleep(2)
    
        show_cars=driver.find_element_by_xpath('//button[@class="btn _3BmFU AgO6q"]').click()
        time.sleep(2)
        print(f'  Getting list of {body_type_selected} cars')
    
    
        #Scroll to last
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(20)
    
        #Getting Details
        listings=driver.find_elements_by_xpath('//div[@class="align-self-center _2RowP"]')
        print('  No. of lisitings:',len(listings))
    
        #Getting prices
        prices=driver.find_elements_by_xpath('//div[@class="_7udZZ"]')
        for cost in prices:
            price.append(cost.text)
        print(f'  prices of {len(listings)} {body_type_selected} cars listed')   
        time.sleep(10)
    
        #Distance 
        driven_km=driver.find_elements_by_xpath('//ul[@class="bVR0c"]/li[1]')
        for km in driven_km:
            distance_driven.append(km.text)
        print(f'  km driven of each {body_type_selected} in {len(listings)} cars listed')
        time.sleep(10)
    
        #Fuel type
        fuel_types=driver.find_elements_by_xpath('//ul[@class="bVR0c"]/li[3]')
        for types in fuel_types:
            fuel_type.append(types.text)
        print(f'  fuel type of each {body_type_selected} in {len(listings)} cars listed')
        time.sleep(10)
    
        #Transmission
        transmission = driver.find_elements_by_xpath('//p[@class="cvakB"]/span')
        for types in transmission:
            transmission_type.append(types.text)
        print(f'  transmission type of each {body_type_selected} in {len(listings)} cars listed')
        time.sleep(10)

        #version
        temp=[]
        versions_types=driver.find_elements_by_xpath('//p[@class="cvakB"]')
        for versions in versions_types:
            temp.append(versions.text)
    
        for items in temp:
            if 'Manual' in items:
                version.append(items.replace('Manual','').strip())
            elif 'Automatic' in items:
                version.append(items.replace('Automatic','').strip())
            else:
                version.append(items.strip())
        print(f'  version of each {body_type_selected} in {len(listings)} cars listed')
        time.sleep(10)
    
        #year
        car_year=driver.find_elements_by_xpath('//h2[@class="_3FpCg"]')
        for years in car_year:
            num=years.text.split()[0]
            year.append(num)
        print(f'  year of each {body_type_selected} in {len(listings)} cars listed')
        time.sleep(10)
    
    
        #Brand
        car_brands=driver.find_elements_by_xpath('//h2[@class="_3FpCg"]')
        for brands in car_brands:
            name=brands.text.split()[1]
            brand.append(name)
        print(f'  brand of each {body_type_selected} in {len(listings)} cars listed')
        time.sleep(10)
    
        #model
        car_models=driver.find_elements_by_xpath('//h2[@class="_3FpCg"]')
        for models in car_models:
            temp=models.text.split()[2:]
            name=' '.join(temp)
            model.append(name)
    
        #listing body-type
        for num in range(1,len(listings)+1):
            car_type.append(body_type_selected)
        time.sleep(10)
        
        
        print('scrolling to filter')
        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
        time.sleep(10)
        try:
            target=driver.find_element_by_xpath('//span[@class="_1QeRC"]')
            target.click()
            time.sleep(5)
        except ElementClickInterceptedException:
            driver.execute_script("window.scrollTo(document.body.scrollHeight,0)")
            target=driver.find_element_by_xpath('//span[@class="_1QeRC"]')
            target.click()
            time.sleep(5)
        print(f'  Deselected {body_type_selected}')
        print('\n')
    
        for num in range(1,len(listings)+1):
            city=driver.find_element_by_xpath('//p[@class="_1OR7d"]').text
            location.append(city)
            
        i+=1
    
        print(f'''  {len(price), len(distance_driven), len(fuel_type), len(location), len(car_type), len(transmission_type), len(version), 
               len(year), len(brand), len(model)}''')
   
        print('\n'*3)
    
    location_dropdown=driver.find_element_by_xpath('//p[@class="_1OR7d"]').click()
    time.sleep(10)
    count+=1
